In [ ]:
# need to add cite to Generating clickbait spoilers with an ensemble of large language models article
!pip install datasets transformers
!pip install mlflow
# userdata.get('secretName')
# Import the necessary modules
from datasets import load_dataset
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
)

In [ ]:
def prepare_data(args):
    tokenizer = AutoTokenizer.from_pretrained(
        args.model_name,
        model_max_length=args.max_length,
    )

    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True)
    
    data_dir = "/path/to/your/dataset"


    tokenized_dataset = load_dataset('csv', data_files={'train': f'{data_dir}/train.csv', 'test': f'{data_dir}/test.csv'}).map(tokenize_function, batched=True)
    tokenized_dataset = tokenized_dataset.rename_column("bleu", "label")
    return tokenizer, tokenized_dataset

In [ ]:
def train(args, tokenizer, tokenized_clickbaits):
    model = AutoModelForSequenceClassification.from_pretrained(
        args.model_name,
        num_labels=1,
    )

    training_args = TrainingArguments(
        output_dir=args.output_dir,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=args.epochs,
        learning_rate=args.learning_rate,
        per_device_train_batch_size=args.batch_size,
        per_device_eval_batch_size=args.batch_size,
        weight_decay=0.01,
        logging_steps=100,
        push_to_hub=False,
        load_best_model_at_end=True,
        report_to="mlflow",
    )
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_clickbaits["train"],
        eval_dataset=tokenized_clickbaits["validation"],
        tokenizer=tokenizer,
    )
    trainer.train()
    return trainer

In [ ]:
class Args:
    def __init__(self, model_name, output_dir, batch_size, epochs, learning_rate, max_length):
        self.model_name = model_name
        self.output_dir = output_dir
        self.batch_size = batch_size
        self.batch_size = batch_size
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.max_length = max_length

In [ ]:
def main(args):
    tokenizer, tokenized_clickbaits = prepare_data(args)

    trainer = train(args, tokenizer, tokenized_clickbaits)
    trainer.save_model()
    tokenizer.save_pretrained(args.output_dir)

In [ ]:
args = Args(model_name='MateuszW/regressor-deberta-iter1-iter2', output_dir='models/regressor/deberta-finetuned', batch_size = 6, epochs = 3, learning_rate = 2e-6, max_length = 512)

main(args)

In [ ]:
# model usage
outputs = model(**inputs)
predicted_value = outputs.logits.item()
predicted_value